In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import dates, colors
from sunpy.time import TimeRange
from astropy.time import Time

import warnings
warnings.filterwarnings("ignore")

## Read in the flare lists

These are created in `flare_list_analysis.ipynb`

In [123]:
stix_flares = pd.read_csv("stix_flares_c_with_loc.csv")
fermi_flares = pd.read_csv("fermi_flares_20210101_20220601.csv")
goes_flares = pd.read_csv("goes_hek_flares_20210101_20220601.csv")

In [124]:
stix_flares.head(2)

,start_UTC,peak_UTC,end_UTC,duration,GOES_flux,GOES_class,LC0_peak_counts_4sec,lon,rad
0,2021-03-02 23:58:03.152,2021-03-02T23:59:23.152,2021-03-03T00:01:23.152,200,0.000001,C1.3,12799,-127.980800,8.444487e+07
1,2021-03-09 12:22:03.313,2021-03-09T12:31:35.314,2021-03-09T12:36:07.315,844,0.000002,C1.7,4351,-119.367484,9.056191e+07


In [125]:
fermi_flares.head(2)

,flare,date,dur,peak counts,total_counts,sun_det0,sun_det1,sun_det2,sun_det3,t_start,t_peak,t_end,unique_month
0,210115_2358,15-Jan-2021,106,555,12977,n5,n3,n1,n4,2021-01-15 23:58:34,2021-01-15 00:00:12,2021-01-15 00:00:21,2021-01
1,210116_2358,16-Jan-2021,106,543,13410,n1,n3,n0,n5,2021-01-16 23:58:32,2021-01-16 23:59:50,2021-01-16 00:00:18,2021-01


In [126]:
print(len(stix_flares))
print(len(fermi_flares))

2055
502


In [127]:
stix_timeranges = [TimeRange(stix_flares["start_UTC"].values[i], stix_flares["end_UTC"].values[i]) for i in range(len(stix_flares))]

In [128]:
fermi_flares["t_peak"].values[0:10]

array(['2021-01-15 00:00:12', '2021-01-16 23:59:50',
       '2021-01-17 23:58:38', '2021-01-24 23:58:41',
       '2021-01-30 00:00:20', '2021-02-07 00:00:04',
       '2021-02-08 23:59:24', '2021-02-26 00:00:22',
       '2021-02-27 23:58:50', '2021-04-16 22:19:40'], dtype=object)

In [129]:
fermi_flare_compare = []
for i in range(len(stix_flares)):
    dt = np.min(np.abs(pd.to_datetime(fermi_flares["t_peak"].values) - \
                       pd.to_datetime(stix_flares["peak_UTC"][i]))) 
    if dt.total_seconds() <= 10*5:
        #print(i, dt.total_seconds())
        fermi_ind = np.argmin(np.abs(pd.to_datetime(fermi_flares["t_peak"].values) - \
                       pd.to_datetime(stix_flares["peak_UTC"][i]))) 
        fermi_flare_compare.append(fermi_ind)
    else:
        #print(i, "no Fermi flare")
        fermi_flare_compare.append("no flare")

In [130]:
stix_flares["fermi"] = fermi_flare_compare

In [131]:
potential_flares = stix_flares[~stix_flares["fermi"].isin(["no flare"])]

In [132]:
potential_flares.iloc[0]

start_UTC               2021-05-07 18:51:07.200
peak_UTC                2021-05-07T19:00:15.200
end_UTC                 2021-05-07T19:18:55.202
duration                                   1668
GOES_flux                              0.000039
GOES_class                                 M3.9
LC0_peak_counts_4sec                     311295
lon                                  -97.426482
rad                            137254463.509537
fermi                                        15
Name: 4, dtype: object

In [133]:
fermi_flares.iloc[potential_flares.iloc[0]["fermi"]]

flare                   210507_1858
date                     7-May-2021
dur                            2211
peak counts                   71659
total_counts               17656436
sun_det0                         n5
sun_det1                         n3
sun_det2                         n1
sun_det3                         n4
t_start         2021-05-07 18:58:45
t_peak          2021-05-07 18:59:34
t_end           2021-05-07 19:35:36
unique_month                2021-05
Name: 15, dtype: object

In [134]:
len(potential_flares)

75

In [135]:
fermi_info_flares = fermi_flares.iloc[potential_flares["fermi"].values][["t_start", "t_peak", "t_end", "peak counts", "total_counts"]]

In [136]:
fermi_info_flares.head(2)

,t_start,t_peak,t_end,peak counts,total_counts
15,2021-05-07 18:58:45,2021-05-07 18:59:34,2021-05-07 19:35:36,71659,17656436
16,2021-05-08 12:51:09,2021-05-08 12:51:17,2021-05-08 12:57:34,1381,75297


In [137]:
potential_flares["fermi_tstart"] = fermi_info_flares["t_start"].values
potential_flares["fermi_tpeak"] = fermi_info_flares["t_peak"].values
potential_flares["fermi_tend"] = fermi_info_flares["t_end"].values
potential_flares["fermi_peak_counts"] = fermi_info_flares["peak counts"].values
potential_flares["fermi_total_counts"] = fermi_info_flares["total_counts"].values

In [138]:
potential_flares

,start_UTC,peak_UTC,end_UTC,duration,GOES_flux,GOES_class,LC0_peak_counts_4sec,lon,rad,fermi,fermi_tstart,fermi_tpeak,fermi_tend,fermi_peak_counts,fermi_total_counts
4,2021-05-07 18:51:07.200,2021-05-07T19:00:15.200,2021-05-07T19:18:55.202,1668,0.000039,M3.9,311295,-97.426482,1.372545e+08,15,2021-05-07 18:58:45,2021-05-07 18:59:34,2021-05-07 19:35:36,71659,17656436
5,2021-05-08 12:43:31.207,2021-05-08T12:51:39.208,2021-05-08T13:09:03.210,1532,0.000002,C1.5,3455,-97.442364,1.375481e+08,16,2021-05-08 12:51:09,2021-05-08 12:51:17,2021-05-08 12:57:34,1381,75297
7,2021-05-09 04:32:15.303,2021-05-09T04:34:47.303,2021-05-09T04:38:51.303,396,0.000002,C1.7,6399,-97.458583,1.378009e+08,18,2021-05-09 04:34:02,2021-05-09 04:35:03,2021-05-09 04:36:54,2720,56128
8,2021-05-09 13:50:15.359,2021-05-09T13:55:11.360,2021-05-09T14:06:59.361,1004,0.000004,C4.0,13823,-97.469042,1.379464e+08,19,2021-05-09 13:51:52,2021-05-09 13:54:22,2021-05-09 13:59:38,11728,944357
13,2021-05-21 19:23:42.830,2021-05-21T19:27:30.831,2021-05-21T19:30:22.831,400,0.000005,C4.8,25599,-98.068510,1.413552e+08,20,2021-05-21 19:24:58,2021-05-21 19:27:58,2021-05-21 19:30:17,3508,105765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,2022-05-18 08:15:31.745,2022-05-18T08:21:35.746,2022-05-18T08:27:19.747,708,0.000003,C2.9,735,165.393904,1.265210e+08,463,2022-05-18 08:21:08,2022-05-18 08:21:44,2022-05-18 08:24:00,3828,50972
1867,2022-05-19 06:46:43.786,2022-05-19T07:03:55.787,2022-05-19T07:43:15.791,3392,0.000056,M5.6,14847,165.393293,1.275486e+08,470,2022-05-19 07:01:35,2022-05-19 07:04:15,2022-05-19 07:07:15,3433,160509
1879,2022-05-20 00:08:59.894,2022-05-20T00:10:19.894,2022-05-20T00:13:59.895,300,0.000004,C4.0,303,165.383332,1.283257e+08,476,2022-05-20 00:09:07,2022-05-20 00:10:47,2022-05-20 00:16:33,6858,903195
1900,2022-05-20 12:24:59.969,2022-05-20T12:28:03.969,2022-05-20T12:33:59.970,540,0.000003,C2.8,2687,165.371492,1.288663e+08,479,2022-05-20 12:28:08,2022-05-20 12:28:34,2022-05-20 12:28:56,6081,81037


In [141]:
potential_flares["goes_class_ind"] = [x[0] for x in potential_flares["GOES_class"]]

In [144]:
potential_flares[potential_flares["goes_class_ind"].isin(["X", "M"])]

,start_UTC,peak_UTC,end_UTC,duration,GOES_flux,GOES_class,LC0_peak_counts_4sec,lon,rad,fermi,fermi_tstart,fermi_tpeak,fermi_tend,fermi_peak_counts,fermi_total_counts,goes_class_ind
4,2021-05-07 18:51:07.200,2021-05-07T19:00:15.200,2021-05-07T19:18:55.202,1668,0.000039,M3.9,311295,-97.426482,1.372545e+08,15,2021-05-07 18:58:45,2021-05-07 18:59:34,2021-05-07 19:35:36,71659,17656436,M
160,2021-10-26 02:44:51.577,2021-10-26T02:46:35.577,2021-10-26T02:51:07.578,376,0.000013,M1.3,172031,-3.798244,1.177350e+08,79,2021-10-26 02:43:37,2021-10-26 02:45:58,2021-10-26 02:54:12,57574,6421385,M
260,2021-11-09 16:37:31.714,2021-11-09T16:53:03.716,2021-11-09T17:30:11.720,3160,0.000020,M2.0,55295,0.004987,1.320723e+08,114,2021-11-09 16:33:26,2021-11-09 16:53:22,2021-11-09 17:32:17,3945,1471584,M
598,2021-12-28 03:58:04.114,2021-12-28T04:01:16.115,2021-12-28T04:10:36.116,752,0.000019,M1.9,94207,-10.278315,1.503632e+08,167,2021-12-28 03:56:31,2021-12-28 04:00:45,2021-12-28 04:17:00,18084,1498985,M
606,2021-12-28 16:05:28.188,2021-12-28T16:10:16.189,2021-12-28T16:20:52.190,924,0.000016,M1.6,94207,-10.445902,1.502300e+08,170,2021-12-28 16:09:33,2021-12-28 16:09:45,2021-12-28 16:48:03,23206,1430046,M
649,2022-01-20 05:54:27.888,2022-01-20T05:58:55.888,2022-01-20T06:11:31.890,1024,0.000055,M5.5,475135,-16.871775,1.382928e+08,191,2022-01-20 05:47:43,2022-01-20 05:58:25,2022-01-20 06:13:58,196274,65107740,M
1867,2022-05-19 06:46:43.786,2022-05-19T07:03:55.787,2022-05-19T07:43:15.791,3392,0.000056,M5.6,14847,165.393293,1.275486e+08,470,2022-05-19 07:01:35,2022-05-19 07:04:15,2022-05-19 07:07:15,3433,160509,M


In [145]:
potential_flares.to_csv("stix_fermi_potential_flares_20210101_20220601.csv", index_label=False)

In [146]:
ls

README.md
combining_flare_lists.ipynb
fermi_flares_20210101_20220601.csv
fermi_gbm_flare_list.txt
flare_list_analysis.ipynb
goes_flares_20210101_20220718.csv
goes_hek_flares_20210101_20220601.csv
matplotlibrc
orbit_from_earth.png
orbit_from_earth_within5.png
orbit_from_earth_within90.png
solo_position_analysis.ipynb
stix_fermi_potential_flares_20210101_20220601.csv
stix_flares_c_with_loc.csv
swpc_flare.py
swpc_flarelist_20100101-20220717_concat.csv
